# FastAPI upload server (payload_video.ipynb)

Notebook ini menyediakan server FastAPI yang menerima upload video (multipart) di `/upload_file` dan menerima JSON payload di `/upload`.

Langkah eksekusi:
1. Jalankan cell instalasi dependensi
2. Jalankan cell setup direktori
3. Jalankan cell definisi server
4. Jalankan cell start server (ngrok akan dicoba jika tersedia)

Hasil: file yang diupload akan disimpan di folder `uploads/` dan payload JSON yang dikirim ke `/upload` akan disimpan di `received_payloads/`. Video akan diproses dengan Whisper untuk speech-to-text.

In [1]:
# Install dependencies (jalankan sekali)
!pip install --quiet fastapi uvicorn nest-asyncio pyngrok python-multipart
!pip install --quiet openai-whisper
!pip install --quiet tqdm
!pip install --quiet imageio-ffmpeg

print('\n✅ All packages installed successfully')


✅ All packages installed successfully


In [2]:
# Siapkan direktori untuk upload dan payload & CONFIGURE FFMPEG
import os
import sys
import shutil

# CRITICAL: Setup ffmpeg FIRST before importing whisper
try:
    import imageio_ffmpeg
    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()
    
    print('✅ ffmpeg from imageio-ffmpeg:')
    print(f'   Executable: {ffmpeg_exe}')
    print(f'   Exists: {os.path.exists(ffmpeg_exe)}')
    
    # Test ffmpeg works
    import subprocess
    result = subprocess.run([ffmpeg_exe, '-version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        version = result.stdout.split('\n')[0]
        print(f'   {version}')
    else:
        raise Exception('ffmpeg test failed')
    
    # WORKAROUND: Copy ffmpeg to a simpler path that Whisper can find
    # Create a local bin directory
    local_bin = os.path.join(os.getcwd(), 'bin')
    os.makedirs(local_bin, exist_ok=True)
    
    # Determine platform-specific executable name
    ffmpeg_name = 'ffmpeg.exe' if os.name == 'nt' else 'ffmpeg'
    local_ffmpeg = os.path.join(local_bin, ffmpeg_name)
    
    # Copy if not exists or different
    if not os.path.exists(local_ffmpeg) or os.path.getsize(local_ffmpeg) != os.path.getsize(ffmpeg_exe):
        print(f'\n📋 Copying ffmpeg to local bin directory...')
        shutil.copy2(ffmpeg_exe, local_ffmpeg)
        print(f'   Copied to: {local_ffmpeg}')
    
    # Set multiple environment variables for maximum compatibility
    os.environ['FFMPEG_BINARY'] = local_ffmpeg
    os.environ['IMAGEIO_FFMPEG_EXE'] = local_ffmpeg
    os.environ['PATH'] = local_bin + os.pathsep + os.environ.get('PATH', '')
    
    # Verify the copied ffmpeg works
    result2 = subprocess.run([local_ffmpeg, '-version'], capture_output=True, text=True, timeout=5)
    if result2.returncode == 0:
        print(f'\n✅ Local ffmpeg is working correctly!')
        print(f'   Location: {local_ffmpeg}')
        print(f'   Added to PATH: {local_bin}')
    else:
        raise Exception('Local ffmpeg copy test failed')
        
except Exception as e:
    print(f'❌ Error setting up ffmpeg: {e}')
    import traceback
    traceback.print_exc()
    sys.exit(1)

# Now setup directories
ROOT_DIR = os.getcwd()
UPLOAD_DIR = os.path.join(ROOT_DIR, 'uploads')
PAYLOAD_DIR = os.path.join(ROOT_DIR, 'received_payloads')
TRANSCRIPTION_DIR = os.path.join(ROOT_DIR, 'transcriptions')
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(PAYLOAD_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)

print('\n📁 Directories:')
print(f'   Upload: {UPLOAD_DIR}')
print(f'   Payload: {PAYLOAD_DIR}')
print(f'   Transcription: {TRANSCRIPTION_DIR}')

# Store ffmpeg path globally for later use
FFMPEG_BINARY = local_ffmpeg
print(f'\n🎯 FFMPEG_BINARY set to: {FFMPEG_BINARY}')

✅ ffmpeg from imageio-ffmpeg:
   Executable: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\.venv\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe
   Exists: True
   ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers

✅ Local ffmpeg is working correctly!
   Location: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\bin\ffmpeg.exe
   Added to PATH: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\bin

📁 Directories:
   Upload: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\uploads
   Payload: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\received_payloads
   Transcription: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\transcriptions

🎯 FFMPEG_BINARY set to: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\bin\ffmpeg.exe


In [3]:
# Definisikan FastAPI app dengan endpoint /upload_file, /upload, /delete_file dan form tester /upload_form
from fastapi import FastAPI, UploadFile, File, Request, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.staticfiles import StaticFiles
import uuid, shutil, json, os, sys
from datetime import datetime, timezone
import urllib.request
import tempfile
from tqdm import tqdm
import hashlib
import time
from urllib.parse import urlparse
import subprocess
from typing import List

# Verify ffmpeg is still in environment
import imageio_ffmpeg
FFMPEG_PATH = imageio_ffmpeg.get_ffmpeg_exe()
if not os.path.exists(FFMPEG_PATH):
    print(f'❌ ERROR: ffmpeg not found at {FFMPEG_PATH}')
    print('Please restart kernel and run cells in order')
    sys.exit(1)
print(f'✅ ffmpeg available: {FFMPEG_PATH}')

# NOW import whisper (after ffmpeg is configured)
import whisper

app = FastAPI(title='AI Interview Upload Server (notebook)')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Mount static folders
app.mount('/uploads', StaticFiles(directory=UPLOAD_DIR), name='uploads')
app.mount('/payloads', StaticFiles(directory=PAYLOAD_DIR), name='payloads')
app.mount('/transcriptions', StaticFiles(directory=TRANSCRIPTION_DIR), name='transcriptions')

# Load Whisper model - UPGRADED to medium for better accuracy
print('\n📥 Loading Whisper model...')
print('ℹ️  Using "medium" model for better accuracy (English)')
print('   This may take a few minutes to download on first run...')
whisper_model = whisper.load_model('medium')  # Changed from 'base' to 'medium'
print('✅ Whisper "medium" model loaded successfully')
print('   Expected accuracy: ~95% for clear English speech\n')

# global state: simpan info payload terakhir yang diterima
last_payload_info = None  # akan di-set ketika /upload dipanggil

# Cache untuk mencegah duplikasi request - gunakan lock untuk thread safety
import threading
request_lock = threading.Lock()
request_cache = {}  # {hash: (timestamp, result)}
CACHE_DURATION = 300  # 5 menit

def get_request_hash(payload):
    """Generate hash dari payload untuk deteksi duplikasi"""
    payload_str = json.dumps(payload, sort_keys=True)
    return hashlib.md5(payload_str.encode()).hexdigest()

def get_local_file_path(url):
    """Extract local file path from URL if it's a local upload"""
    try:
        parsed = urlparse(url)
        # Check if URL is pointing to local uploads
        if '/uploads/' in parsed.path:
            filename = parsed.path.split('/uploads/')[-1]
            local_path = os.path.join(UPLOAD_DIR, filename)
            if os.path.exists(local_path):
                return local_path
    except Exception as e:
        print(f'Error parsing URL: {e}')
    return None

def transcribe_video(video_path):
    """Melakukan speech-to-text pada video menggunakan Whisper dengan akurasi tinggi"""
    try:
        # Verify file exists and is readable
        if not os.path.exists(video_path):
            raise Exception(f"Video file not found: {video_path}")
        
        if not os.access(video_path, os.R_OK):
            raise Exception(f"Video file is not readable: {video_path}")
        
        # Get file info
        file_size = os.path.getsize(video_path) / (1024 * 1024)  # MB
        print(f'📁 Video: {os.path.basename(video_path)} ({file_size:.2f} MB)')
        
        # Ensure ffmpeg is available
        if not os.path.exists(FFMPEG_BINARY):
            raise Exception(f"ffmpeg not found: {FFMPEG_BINARY}")
        
        # Set ffmpeg explicitly
        os.environ['FFMPEG_BINARY'] = FFMPEG_BINARY
        print(f'🔧 Using ffmpeg: {FFMPEG_BINARY}')
        
        # Transcribe with optimized parameters for English accuracy
        print('🔄 Starting high-accuracy Whisper transcription...')
        print('   Model: medium (better than base)')
        print('   Language: English (en)')
        print('   Processing...')
        
        result = whisper_model.transcribe(
            video_path,
            # Core parameters for accuracy
            language='en',  # Changed to English
            task='transcribe',
            
            # Quality parameters
            fp16=False,  # Use full precision for better accuracy
            temperature=0.0,  # Deterministic output (most likely transcription)
            
            # Advanced parameters for better accuracy
            beam_size=5,  # Beam search for better results (default is 5)
            best_of=5,  # Number of candidates when sampling (higher = better but slower)
            patience=1.0,  # Beam search patience factor
            
            # Probability thresholds (tuned for English)
            compression_ratio_threshold=2.4,  # Detect repetitions
            logprob_threshold=-1.0,  # Keep most transcriptions
            no_speech_threshold=0.6,  # Detect silence segments
            
            # Conditioning parameters
            condition_on_previous_text=True,  # Use context for better accuracy
            initial_prompt="This is an interview conversation in English.",  # Context hint
            
            # Output control
            verbose=False,
            word_timestamps=False  # Disable for faster processing (enable if needed)
        )
        
        # Validate result
        if not result or 'text' not in result:
            raise Exception("Whisper returned invalid result")
        
        text = result['text'].strip()
        
        if not text:
            print('⚠️  No speech detected in video')
            return "[No speech detected in video]"
        
        # Quality metrics (if available)
        if 'language' in result:
            detected_lang = result['language']
            print(f'   Detected language: {detected_lang}')
            if detected_lang != 'en':
                print(f'   ⚠️  Warning: Detected language is not English!')
        
        print(f'✅ Transcription completed successfully')
        print(f'   Length: {len(text)} characters')
        print(f'   Words: ~{len(text.split())} words')
        print(f'   Preview: "{text[:150]}..."')
        
        return text
            
    except Exception as e:
        # Log full error for debugging
        import traceback
        error_detail = traceback.format_exc()
        print(f'❌ Transcription error details:\n{error_detail}')
        raise Exception(f"Transcription failed: {str(e)}")

def download_video(url, dest_path):
    """Download video dari URL dengan progress bar"""
    try:
        print(f'Downloading video from: {url}')
        
        # Download dengan progress bar
        def reporthook(block_num, block_size, total_size):
            if not hasattr(reporthook, 'pbar'):
                reporthook.pbar = tqdm(total=total_size, unit='B', unit_scale=True, desc='Download')
            downloaded = block_num * block_size
            if downloaded < total_size:
                reporthook.pbar.update(block_size)
            else:
                reporthook.pbar.close()
                delattr(reporthook, 'pbar')
        
        urllib.request.urlretrieve(url, dest_path, reporthook)
        print('Download completed!')
        return True
    except Exception as e:
        raise Exception(f"Download error: {str(e)}")

@app.post('/upload_file')
async def upload_file(request: Request, file: UploadFile = File(...)):
    try:
        # simpan file dengan nama unik
        ext = os.path.splitext(file.filename)[1] or ''
        safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
        dest_path = os.path.join(UPLOAD_DIR, safe_name)
        with open(dest_path, 'wb') as buffer:
            shutil.copyfileobj(file.file, buffer)
        base_url = str(request.base_url).rstrip('/')
        file_url = f"{base_url}/uploads/{safe_name}"
        # kembalikan url dan safe filename agar client bisa menghapus jika perlu
        return JSONResponse({'success': True, 'url': file_url, 'name': safe_name})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.delete('/delete_file')
async def delete_file(payload: dict):
    try:
        # payload: { 'name': '<safe_name>' }
        name = payload.get('name') if isinstance(payload, dict) else None
        if not name:
            raise HTTPException(status_code=400, detail='Missing file name')
        file_path = os.path.join(UPLOAD_DIR, name)
        if os.path.exists(file_path):
            os.remove(file_path)
            return JSONResponse({'success': True, 'deleted': name})
        else:
            return JSONResponse({'success': False, 'error': 'file not found'}, status_code=404)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post('/upload')
async def receive_videos_and_process(
    request: Request,
    candidate_name: str = File(...),
    videos: List[UploadFile] = File(...)
):
    """
    Upload multiple videos, auto-build JSON payload, and process transcription.
    
    Workflow:
    1. Upload semua video ke server
    2. Build JSON payload otomatis
    3. Proses transcription untuk setiap video
    4. Return hasil + redirect
    """
    print(f'\n🔵 NEW UPLOAD REQUEST RECEIVED')
    print(f'   Candidate: {candidate_name}')
    print(f'   Videos: {len(videos)} file(s)')
    
    try:
        # 1. Upload semua video ke server dan dapatkan URLs
        base_url = str(request.base_url).rstrip('/')
        uploaded_videos = []
        
        print(f'\n📤 Uploading {len(videos)} video(s)...')
        for idx, video in enumerate(videos, 1):
            try:
                # Save video dengan nama unik
                ext = os.path.splitext(video.filename)[1] or '.webm'
                safe_name = f"{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}{ext}"
                dest_path = os.path.join(UPLOAD_DIR, safe_name)
                
                print(f'   Uploading video {idx}/{len(videos)}: {video.filename}')
                with open(dest_path, 'wb') as buffer:
                    shutil.copyfileobj(video.file, buffer)
                
                file_url = f"{base_url}/uploads/{safe_name}"
                uploaded_videos.append({
                    'positionId': idx,
                    'isVideoExist': True,
                    'recordedVideoUrl': file_url,
                    'filename': safe_name
                })
                print(f'   ✅ Uploaded: {file_url}')
                
            except Exception as e:
                print(f'   ❌ Failed to upload video {idx}: {str(e)}')
                uploaded_videos.append({
                    'positionId': idx,
                    'isVideoExist': False,
                    'recordedVideoUrl': None,
                    'error': str(e)
                })
        
        # 2. Build JSON payload otomatis
        payload = {
            'success': True,
            'data': {
                'candidate': {
                    'name': candidate_name
                },
                'reviewChecklists': {
                    'project': [],
                    'interviews': uploaded_videos
                }
            }
        }
        
        # Simpan payload ke file
        fname = f"payload_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.json"
        dest = os.path.join(PAYLOAD_DIR, fname)
        with open(dest, 'w', encoding='utf-8') as f:
            json.dump(payload, f, ensure_ascii=False, indent=2)
        
        payload_url = f"{base_url}/payloads/{fname}"
        print(f'\n💾 Payload saved: {payload_url}')
        
        # 3. Proses transcription untuk setiap video
        transcriptions = []
        
        print(f'\n{"="*60}')
        print(f'🎙️  Processing transcriptions for: {candidate_name}')
        print(f'   Total videos: {len(uploaded_videos)}')
        print(f'{"="*60}\n')
        
        for interview in uploaded_videos:
            if not interview.get('isVideoExist') or not interview.get('recordedVideoUrl'):
                transcriptions.append({
                    'positionId': interview['positionId'],
                    'error': interview.get('error', 'Video upload failed')
                })
                continue
            
            position_id = interview['positionId']
            video_url = interview['recordedVideoUrl']
            
            try:
                print(f'\n{"─"*60}')
                print(f'📹 Processing Video {position_id}/{len(uploaded_videos)}')
                print(f'   Position ID: {position_id}')
                print(f'   URL: {video_url}')
                print(f'{"─"*60}')
                
                # Get local file path
                local_file = get_local_file_path(video_url)
                
                if not local_file:
                    raise Exception(f"Local file not found for URL: {video_url}")
                
                # Transcribe
                print(f'\n🎤 Transcribing video for position {position_id}...')
                transcription_text = transcribe_video(local_file)
                
                # Save transcription
                trans_fname = f"transcription_pos{position_id}_{datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex}.txt"
                trans_path = os.path.join(TRANSCRIPTION_DIR, trans_fname)
                with open(trans_path, 'w', encoding='utf-8') as f:
                    f.write(f"Candidate: {candidate_name}\n")
                    f.write(f"Position ID: {position_id}\n")
                    f.write(f"Video URL: {video_url}\n")
                    f.write(f"Transcribed at: {datetime.now(timezone.utc).isoformat()}\n")
                    f.write(f"\n{'='*50}\n\n")
                    f.write(transcription_text)
                
                transcription_url = f"{base_url}/transcriptions/{trans_fname}"
                print(f'\n✅ Transcription saved: {transcription_url}')
                
                transcriptions.append({
                    'positionId': position_id,
                    'videoUrl': video_url,
                    'transcription': transcription_text,
                    'transcriptionUrl': transcription_url,
                    'transcriptionFile': trans_fname
                })
                
            except Exception as e:
                error_msg = f"Error processing video for position {position_id}: {str(e)}"
                print(f'\n❌ {error_msg}\n')
                transcriptions.append({
                    'positionId': position_id,
                    'videoUrl': video_url,
                    'error': error_msg
                })
        
        print(f'\n{"="*60}')
        print(f'✅ Completed! Processed {len(transcriptions)} video(s)')
        print(f'{"="*60}\n')
        
        # 4. Update global state
        global last_payload_info
        last_payload_info = {
            'name': fname,
            'url': payload_url,
            'saved_at': datetime.now(timezone.utc).isoformat(),
            'candidate_name': candidate_name,
            'transcriptions': transcriptions
        }
        
        # 5. Return response
        dashboard_url = "halaman_dasboard.html"
        response_data = {
            'success': True,
            'payload': payload,
            'saved_as': fname,
            'url': payload_url,
            'redirect': dashboard_url,
            'processed_videos': len(transcriptions),
            'transcriptions': transcriptions
        }
        
        return JSONResponse(response_data)
        
    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f'❌ Error in upload endpoint:\n{error_detail}')
        raise HTTPException(status_code=500, detail=str(e))

# Endpoint untuk mengambil info payload terakhir yang diterima
@app.get('/last_payload')
async def get_last_payload():
    if last_payload_info is None:
        return JSONResponse({'success': False, 'message': 'No payload received yet'}, status_code=404)
    return JSONResponse({'success': True, 'last_payload': last_payload_info})

# Endpoint untuk membaca isi payload terakhir (jika ada)
@app.get('/last_payload/content')
async def get_last_payload_content():
    if last_payload_info is None:
        return JSONResponse({'success': False, 'message': 'No payload received yet'}, status_code=404)
    fp = os.path.join(PAYLOAD_DIR, last_payload_info['name'])
    if not os.path.exists(fp):
        return JSONResponse({'success': False, 'message': 'File not found'}, status_code=404)
    try:
        with open(fp, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return JSONResponse({'success': True, 'content': data})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/')
async def index():
    return {'message': 'AI Interview Upload Server (notebook) running. Use /upload_file and /upload endpoints.'}

# simple HTML form for manual testing (GET).
@app.get('/upload_form')
async def upload_form():
    html = '''
    <html>
      <head><meta charset="utf-8"><title>Upload Form</title></head>
      <body>
        <h3>Test upload to /upload_file (multipart POST)</h3>
        <form action="/upload_file" enctype="multipart/form-data" method="post">
          <input name="file" type="file" accept="video/*" required />
          <button type="submit">Upload</button>
        </form>
        <p>Setelah submit, server akan menyimpan file dan mengembalikan JSON berisi URL file.</p>
      </body>
    </html>
    '''
    return HTMLResponse(content=html, status_code=200)

✅ ffmpeg available: d:\ASAH\CAPSTONE_V2\Interview_Assesment_System\bin\ffmpeg.exe

📥 Loading Whisper model...
ℹ️  Using "medium" model for better accuracy (English)
   This may take a few minutes to download on first run...


100%|█████████████████████████████████████| 1.42G/1.42G [02:59<00:00, 8.51MiB/s]


✅ Whisper "medium" model loaded successfully
   Expected accuracy: ~95% for clear English speech



In [4]:
# Jalankan server uvicorn di dalam notebook (tanpa ngrok)
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()
PORT = 8888

# Hentikan server sebelumnya jika ada
if 'server_thread' in globals() and server_thread is not None:
    try:
        print('⏸️  Stopping previous server...')
        if 'server' in globals() and server is not None:
            server.should_exit = True
        # Tunggu thread selesai (dengan timeout)
        if server_thread.is_alive():
            server_thread.join(timeout=2)
        print('✅ Previous server stopped.')
    except Exception as e:
        print(f'⚠️  Error stopping previous server: {e}')

# Buat server instance baru dengan log level yang lebih rendah
config = uvicorn.Config(
    app=app, 
    host='0.0.0.0', 
    port=PORT, 
    log_level='warning',  # Kurangi verbosity untuk menghindari duplikasi log
    access_log=False  # Nonaktifkan access log di console
)
server = uvicorn.Server(config=config)

# Fungsi untuk menjalankan server di thread
def run_server_in_thread():
    import asyncio
    # Buat event loop baru untuk thread ini
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f'❌ Server error: {e}')
    finally:
        loop.close()

# Jalankan server di background thread
server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
server_thread.start()

print('━' * 60)
print('🚀 Server started successfully!')
print(f'📍 Local URL: http://127.0.0.1:{PORT}')
print(f'📍 Network URL: http://0.0.0.0:{PORT}')
print(f'🔧 Endpoints:')
print(f'   - POST /upload_file  (upload video file)')
print(f'   - POST /upload       (send JSON payload & process videos)')
print(f'   - DELETE /delete_file (delete uploaded file)')
print(f'   - GET  /upload_form  (test form)')
print('ℹ️  Use Interrupt Kernel to stop the server')
print('━' * 60)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🚀 Server started successfully!
📍 Local URL: http://127.0.0.1:8888
📍 Network URL: http://0.0.0.0:8888
🔧 Endpoints:
   - POST /upload_file  (upload video file)
   - POST /upload       (send JSON payload & process videos)
   - DELETE /delete_file (delete uploaded file)
   - GET  /upload_form  (test form)
ℹ️  Use Interrupt Kernel to stop the server
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Catatan:
- Akses lokal: http://127.0.0.1:8888
- Untuk menguji upload melalui browser buka: http://127.0.0.1:8888/upload_form
- Jika ngrok berhasil, public URL akan dicetak dan file yang diupload tersedia di <public_url>/uploads/<filename>
- Payload yang diterima melalui `/upload` disimpan di folder `received_payloads/`.
- Video akan diproses menggunakan Whisper dan transkripsi disimpan di folder `transcriptions/`.
- Server akan memproses semua video dalam array `data.reviewChecklists.interviews`.
- Format payload yang diterima:
  ```json
  {
    "success": true,
    "data": {
      "candidate": { "name": "nama_kandidat" },
      "reviewChecklists": {
        "interviews": [
          {
            "positionId": 1,
            "isVideoExist": true,
            "recordedVideoUrl": "http://..."
          }
        ]
      }
    }
  }
  ```